In [64]:
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql.functions import col, expr, concat_ws, lit, regexp_extract, regexp_replace, asc, desc
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean, round, month, year, quarter, date_format, when

from pydataset import data


import warnings
warnings.filterwarnings('ignore')

### 1. Create a spark data frame that contains your favorite programming languages.

    - The name of the column should be language
    - View the schema of the dataframe
    - Output the shape of the dataframe
    - Show the first 5 records in the dataframe

In [2]:
programming_df = pd.DataFrame({'name': ['python', 'sql', 'regex', 'java', 'c++']})

In [3]:
programming_df

,name
0,python
1,sql
2,regex
3,java
4,c++


In [4]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 15:01:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.createDataFrame(programming_df)
df

DataFrame[name: string]

In [6]:
df.printSchema()


root
 |-- name: string (nullable = true)



In [7]:
df.count()
print(df.count(),'rows', len(df.columns), 'columns')

5 rows 1 columns


In [8]:
df.show(5)

+------+
|  name|
+------+
|python|
|   sql|
| regex|
|  java|
|   c++|
+------+



### 2. Load the mpg dataset as a spark dataframe.

    a. Create 1 column of output that contains a message like the one below:
        
            - The 1999 audi a4 has a 4 cylinder engine.

        For each vehicle.

    b. Transform the trans column so that it only contains either manual or auto.

In [9]:
mpg = data('mpg')

In [10]:
df = data('mpg')
df = spark.createDataFrame(df)
df

DataFrame[manufacturer: string, model: string, displ: double, year: bigint, cyl: bigint, trans: string, drv: string, cty: bigint, hwy: bigint, fl: string, class: string]

In [11]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [12]:
mpg_df = df.withColumn(
    "output",
    concat_ws(" ", 
              lit("- The"), 
              df.year, 
              df.manufacturer, 
              df.model, 
              lit("has a"), 
              df.cyl, 
              lit("cylinder engine.")
             )
)


mpg_df.select(mpg_df.output).show(truncate = False)


+----------------------------------------------------------------+
|output                                                          |
+----------------------------------------------------------------+
|- The 1999 audi a4 has a 4 cylinder engine.                     |
|- The 1999 audi a4 has a 4 cylinder engine.                     |
|- The 2008 audi a4 has a 4 cylinder engine.                     |
|- The 2008 audi a4 has a 4 cylinder engine.                     |
|- The 1999 audi a4 has a 6 cylinder engine.                     |
|- The 1999 audi a4 has a 6 cylinder engine.                     |
|- The 2008 audi a4 has a 6 cylinder engine.                     |
|- The 1999 audi a4 quattro has a 4 cylinder engine.             |
|- The 1999 audi a4 quattro has a 4 cylinder engine.             |
|- The 2008 audi a4 quattro has a 4 cylinder engine.             |
|- The 2008 audi a4 quattro has a 4 cylinder engine.             |
|- The 1999 audi a4 quattro has a 6 cylinder engine.          

In [13]:
df.select(df.trans).show()

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
|manual(m5)|
|  auto(av)|
|manual(m5)|
|  auto(l5)|
|manual(m6)|
|  auto(s6)|
|  auto(l5)|
|manual(m5)|
|  auto(s6)|
|manual(m6)|
|  auto(l5)|
|  auto(s6)|
|  auto(s6)|
|  auto(l4)|
|  auto(l4)|
+----------+
only showing top 20 rows



In [14]:
df.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [15]:
df = df.withColumn("trans", regexp_replace("trans", r"\(.*\)", ""))
df = df.withColumn("trans", regexp_replace("trans", r"auto.*", "auto"))
df = df.withColumn("trans", regexp_replace("trans", r"manual.*", "manual"))


In [16]:
df.show()

+------------+------------------+-----+----+---+------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl| trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto|  4| 16| 25|  p|compact|
|        audi|        a4 quattro|  2.0|2

In [17]:
df.select("trans", 
          regexp_replace("trans", r"\(.*\)", "").alias('transmission')
         ).show()

+------+------------+
| trans|transmission|
+------+------------+
|  auto|        auto|
|manual|      manual|
|manual|      manual|
|  auto|        auto|
|  auto|        auto|
|manual|      manual|
|  auto|        auto|
|manual|      manual|
|  auto|        auto|
|manual|      manual|
|  auto|        auto|
|  auto|        auto|
|manual|      manual|
|  auto|        auto|
|manual|      manual|
|  auto|        auto|
|  auto|        auto|
|  auto|        auto|
|  auto|        auto|
|  auto|        auto|
+------+------------+
only showing top 20 rows



### 3. Load the tips dataset as a spark dataframe.

    a. What percentage of observations are smokers?
    b. Create a column that contains the tip percentage
    c. Calculate the average tip percentage for each combination of sex and smoker.

In [18]:
tips = data('tips')

In [19]:
tips_df = spark.createDataFrame(tips)
tips_df

DataFrame[total_bill: double, tip: double, sex: string, smoker: string, day: string, time: string, size: bigint]

In [20]:
tips_df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [21]:
tips_df.describe().show()

+-------+------------------+------------------+------+------+----+------+------------------+
|summary|        total_bill|               tip|   sex|smoker| day|  time|              size|
+-------+------------------+------------------+------+------+----+------+------------------+
|  count|               244|               244|   244|   244| 244|   244|               244|
|   mean|19.785942622950813|2.9982786885245907|  null|  null|null|  null| 2.569672131147541|
| stddev| 8.902411954856856| 1.383638189001182|  null|  null|null|  null|0.9510998047322344|
|    min|              3.07|               1.0|Female|    No| Fri|Dinner|                 1|
|    max|             50.81|              10.0|  Male|   Yes|Thur| Lunch|                 6|
+-------+------------------+------------------+------+------+----+------+------------------+



In [22]:
tips_df.filter(tips_df.smoker == 'Yes').show()

+----------+----+------+------+----+------+----+
|total_bill| tip|   sex|smoker| day|  time|size|
+----------+----+------+------+----+------+----+
|     38.01| 3.0|  Male|   Yes| Sat|Dinner|   4|
|     11.24|1.76|  Male|   Yes| Sat|Dinner|   2|
|     20.29|3.21|  Male|   Yes| Sat|Dinner|   2|
|     13.81| 2.0|  Male|   Yes| Sat|Dinner|   2|
|     11.02|1.98|  Male|   Yes| Sat|Dinner|   2|
|     18.29|3.76|  Male|   Yes| Sat|Dinner|   4|
|      3.07| 1.0|Female|   Yes| Sat|Dinner|   1|
|     15.01|2.09|  Male|   Yes| Sat|Dinner|   2|
|     26.86|3.14|Female|   Yes| Sat|Dinner|   2|
|     25.28| 5.0|Female|   Yes| Sat|Dinner|   2|
|     17.92|3.08|  Male|   Yes| Sat|Dinner|   2|
|     19.44| 3.0|  Male|   Yes|Thur| Lunch|   2|
|     32.68| 5.0|  Male|   Yes|Thur| Lunch|   2|
|     28.97| 3.0|  Male|   Yes| Fri|Dinner|   2|
|      5.75| 1.0|Female|   Yes| Fri|Dinner|   2|
|     16.32| 4.3|Female|   Yes| Fri|Dinner|   2|
|     40.17|4.73|  Male|   Yes| Fri|Dinner|   4|
|     27.28| 4.0|  M

In [23]:
smoker_pct = tips_df.filter(tips_df.smoker == "Yes").count() / tips_df.count() * 100
print("Percentage of observations that are smokers is {:.2f}%".format(smoker_pct))

Percentage of observations that are smokers is 38.11%


In [24]:
tips_df = tips_df.withColumn("tips_pct", round((col("tip")/col("total_bill"))*100, 2))
tips_df.show()

+----------+----+------+------+---+------+----+--------+
|total_bill| tip|   sex|smoker|day|  time|size|tips_pct|
+----------+----+------+------+---+------+----+--------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|    5.94|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|   16.05|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|   16.66|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|   13.98|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|   14.68|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|   18.62|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|   22.81|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|   11.61|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|   13.03|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|   21.85|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|   16.65|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|   14.18|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|   10.18|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|   16.28|
|     14.83|3.02|Female|    No|

In [25]:
tips_df.groupBy(['sex','smoker']).agg(round(avg("tips_pct"),2).alias('average_pct')).show()

+------+------+-----------+
|   sex|smoker|average_pct|
+------+------+-----------+
|  Male|    No|      16.07|
|Female|    No|      15.69|
|  Male|   Yes|      15.28|
|Female|   Yes|      18.21|
+------+------+-----------+



### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?

In [26]:
from vega_datasets import data

In [27]:
weather = data.seattle_weather()

In [28]:
weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
seattle_df = spark.createDataFrame(weather)
seattle_df.show()

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|
|2012-01-11|          0.0|     6.1|    -1.1| 5.1|    sun|
|2012-01-12|          0.0|     6.1|    -1.7| 1.9|    sun|
|2012-01-13|          0.0|     5.0|    -2.8| 1.3|    sun|
|2012-01-14|          4.1|     4.4|     0.6| 5.3|   snow|
|2012-01-15|  

In [29]:
seattle_df = seattle_df.withColumn("temp_max_f", round(col("temp_max") * 9/5 + 32, 2))\
                     .withColumn("temp_min_f", round(col("temp_min") * 9/5 + 32, 2))\
                     .drop("temp_max", "temp_min")

seattle_df.show()


+----------+-------------+----+-------+----------+----------+
|      date|precipitation|wind|weather|temp_max_f|temp_min_f|
+----------+-------------+----+-------+----------+----------+
|2012-01-01|          0.0| 4.7|drizzle|     55.04|      41.0|
|2012-01-02|         10.9| 4.5|   rain|     51.08|     37.04|
|2012-01-03|          0.8| 2.3|   rain|     53.06|     44.96|
|2012-01-04|         20.3| 4.7|   rain|     53.96|     42.08|
|2012-01-05|          1.3| 6.1|   rain|     48.02|     37.04|
|2012-01-06|          2.5| 2.2|   rain|     39.92|     35.96|
|2012-01-07|          0.0| 2.3|   rain|     44.96|     37.04|
|2012-01-08|          0.0| 2.0|    sun|      50.0|     37.04|
|2012-01-09|          4.3| 3.4|   rain|     48.92|      41.0|
|2012-01-10|          1.0| 3.4|   rain|     42.98|     33.08|
|2012-01-11|          0.0| 5.1|    sun|     42.98|     30.02|
|2012-01-12|          0.0| 1.9|    sun|     42.98|     28.94|
|2012-01-13|          0.0| 1.3|    sun|      41.0|     26.96|
|2012-01

In [30]:
seattle_df.select("date",
                  regexp_extract("date", r"^\d{4}", 0).alias("year"),
                  regexp_extract("date", r"-(\d{2})-", 1).alias("month"),
                  regexp_extract("date", r"-(\d{2})$", 1).alias("day")).show()


+----------+----+-----+---+
|      date|year|month|day|
+----------+----+-----+---+
|2012-01-01|2012|   01| 01|
|2012-01-02|2012|   01| 02|
|2012-01-03|2012|   01| 03|
|2012-01-04|2012|   01| 04|
|2012-01-05|2012|   01| 05|
|2012-01-06|2012|   01| 06|
|2012-01-07|2012|   01| 07|
|2012-01-08|2012|   01| 08|
|2012-01-09|2012|   01| 09|
|2012-01-10|2012|   01| 10|
|2012-01-11|2012|   01| 11|
|2012-01-12|2012|   01| 12|
|2012-01-13|2012|   01| 13|
|2012-01-14|2012|   01| 14|
|2012-01-15|2012|   01| 15|
|2012-01-16|2012|   01| 16|
|2012-01-17|2012|   01| 17|
|2012-01-18|2012|   01| 18|
|2012-01-19|2012|   01| 19|
|2012-01-20|2012|   01| 20|
+----------+----+-----+---+
only showing top 20 rows



In [31]:
seattle_df.withColumn("month", month("date"))\
    .groupBy("month")\
    .agg(round(mean("precipitation"), 2).alias("total_rainfall"))\
    .sort(desc("total_rainfall"))\
    .show(3)


+-----+--------------+
|month|total_rainfall|
+-----+--------------+
|   11|          5.35|
|   12|          5.02|
|    3|          4.89|
+-----+--------------+
only showing top 3 rows



In [32]:
seattle_df.select('weather').distinct().show()

+-------+
|weather|
+-------+
|drizzle|
|   rain|
|    sun|
|   snow|
|    fog|
+-------+



In [33]:
seattle_df.withColumn("year", year("date"))\
    .groupBy("year")\
    .agg(round(mean("wind"), 2).alias("total_wind"))\
    .sort(desc("total_wind"))\
    .show()


+----+----------+
|year|total_wind|
+----+----------+
|2012|       3.4|
|2014|      3.39|
|2015|      3.16|
|2013|      3.02|
+----+----------+



In [34]:
seattle_df.filter(month("date") == 1)\
    .groupBy("weather")\
    .agg(count("weather").alias("freq_weather"))\
    .sort(desc("freq_weather"))\
    .show()


+-------+------------+
|weather|freq_weather|
+-------+------------+
|    fog|          38|
|   rain|          35|
|    sun|          33|
|drizzle|          10|
|   snow|           8|
+-------+------------+



#### Question 4 continued

- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).

In [35]:
seattle_df.filter(month('date') == '07').where(col('weather') == 'sun').show()


+----------+-------------+----+-------+----------+----------+
|      date|precipitation|wind|weather|temp_max_f|temp_min_f|
+----------+-------------+----+-------+----------+----------+
|2012-07-04|          0.0| 3.8|    sun|     69.08|     48.92|
|2012-07-06|          0.0| 2.1|    sun|      77.0|     51.98|
|2012-07-07|          0.0| 3.8|    sun|     80.06|     55.04|
|2012-07-17|          0.0| 2.6|    sun|     71.06|      59.0|
|2012-07-18|          0.0| 2.9|    sun|     69.98|     57.92|
|2012-07-19|          0.0| 2.2|    sun|      77.0|     57.92|
|2012-07-21|          0.0| 2.3|    sun|     75.02|     57.02|
|2012-07-24|          0.0| 4.3|    sun|     73.94|     53.96|
|2012-07-25|          0.0| 2.6|    sun|     80.06|     55.04|
|2012-07-29|          0.0| 2.0|    sun|     73.04|      59.0|
|2012-07-30|          0.0| 3.0|    sun|     66.92|     55.94|
|2012-07-31|          0.0| 2.8|    sun|     73.04|     57.02|
|2013-07-01|          0.0| 2.3|    sun|     89.06|     64.94|
|2013-07

In [39]:
# Filter the DataFrame to include only sunny days in July 2013 and 2014
sunny_days_july_2013_2014 = seattle_df.filter((month('date') == '07') & (year('date').isin(['2013', '2014'])) & (col('weather') == 'sun'))

In [41]:
sunny_days_july_2013_2014.show()

+----------+-------------+----+-------+----------+----------+
|      date|precipitation|wind|weather|temp_max_f|temp_min_f|
+----------+-------------+----+-------+----------+----------+
|2013-07-01|          0.0| 2.3|    sun|     89.06|     64.94|
|2013-07-02|          0.0| 3.0|    sun|     82.94|     60.08|
|2013-07-03|          0.0| 3.2|    sun|     78.98|     62.06|
|2013-07-05|          0.0| 2.6|    sun|     73.94|     57.02|
|2013-07-06|          0.0| 2.2|    sun|     78.98|     55.94|
|2013-07-07|          0.0| 2.9|    sun|     75.02|     57.02|
|2013-07-08|          0.0| 2.8|    sun|     80.06|     55.94|
|2013-07-09|          0.0| 2.5|    sun|      86.0|      59.0|
|2013-07-10|          0.0| 2.6|    sun|     71.96|     57.02|
|2013-07-11|          0.0| 3.0|    sun|     73.04|     53.96|
|2013-07-12|          0.0| 2.2|    sun|     66.92|     55.94|
|2013-07-13|          0.0| 3.1|    sun|     78.98|     51.98|
|2013-07-14|          0.0| 3.0|    sun|     82.04|     55.04|
|2013-07

In [60]:
# Compute the average high and low temperature for sunny days in July 2013 and 2014
avg_temp_max_f = sunny_days_july_2013_2014.agg(avg(col('temp_max_f'))).collect()[0][0]
avg_temp_min_f = sunny_days_july_2013_2014.agg(avg(col('temp_min_f'))).collect()[0][0]

#print(avg_temp_max_f)
#print(avg_temp_min_f)

# Print the results
print(f"Average high temperature on sunny days in July 2013 and 2014 is {avg_temp_max_f:.2f} F")
print(f"Average low temperature on sunny days in July 2013 and 2014 is {avg_temp_min_f:.2f} F")

Average high temperature on sunny days in July 2013 and 2014 is 80.29 F
Average low temperature on sunny days in July 2013 and 2014 is 57.53 F


In [63]:
# Filter the DataFrame to include only days in Q3 of 2015
q3_2015 = seattle_df.filter((year('date') == '2015') & (quarter('date') == '3'))

# Count the number of rainy days in Q3 of 2015
rainy_days_q3_2015 = q3_2015.filter(col('weather') == 'rain').count()

# Count the total number of days in Q3 of 2015
total_days_q3_2015 = q3_2015.count()

# Compute the percentage of days that were rainy in Q3 of 2015
percentage_rainy_q3_2015 = (rainy_days_q3_2015 / total_days_q3_2015) * 100

# Print the result
print(f"Percentage of days that were rainy in Q3 of 2015: {percentage_rainy_q3_2015:.2f}%")


Percentage of days that were rainy in Q3 of 2015: 2.17%


In [65]:
# Group the DataFrame by year and weather, and count the number of days for each group
yearly_weather_counts = seattle_df.groupBy(year('date').alias('year'), 'weather') \
                                  .agg(count('*').alias('count'))

# Pivot the DataFrame so that each weather type becomes a column
yearly_weather_counts_pivot = yearly_weather_counts.groupBy('year') \
                                                  .pivot('weather') \
                                                  .agg(count('count').alias('count')) \
                                                  .fillna(0)

# Compute the percentage of rainy days for each year
yearly_rainy_perc = yearly_weather_counts_pivot.select('year', (when(col('rain') > 0, col('rain')) \
                                                               .otherwise(0) / col('sum(count)')) \
                                                              .alias('percentage_rainy_days'))

# Print the result
yearly_rainy_perc.show()


AnalysisException: Column '`sum(count)`' does not exist. Did you mean one of the following? [sun, snow, fog, rain, drizzle, year];
'Project [year#1699, (CASE WHEN (rain#1702L > cast(0 as bigint)) THEN rain#1702L ELSE cast(0 as bigint) END / 'sum(count)) AS percentage_rainy_days#1711]
+- Project [coalesce(year#1638, cast(0.0 as int)) AS year#1699, coalesce(drizzle#1672L, cast(0.0 as bigint)) AS drizzle#1700L, coalesce(fog#1673L, cast(0.0 as bigint)) AS fog#1701L, coalesce(rain#1674L, cast(0.0 as bigint)) AS rain#1702L, coalesce(snow#1675L, cast(0.0 as bigint)) AS snow#1703L, coalesce(sun#1676L, cast(0.0 as bigint)) AS sun#1704L]
   +- Project [year#1638, __pivot_count(count) AS count AS `count(count) AS count`#1671[0] AS drizzle#1672L, __pivot_count(count) AS count AS `count(count) AS count`#1671[1] AS fog#1673L, __pivot_count(count) AS count AS `count(count) AS count`#1671[2] AS rain#1674L, __pivot_count(count) AS count AS `count(count) AS count`#1671[3] AS snow#1675L, __pivot_count(co

In [67]:
# Pivot the weather column and compute the count of each weather type for each year
yearly_weather_counts = seattle_df.groupBy(year('date').alias('year'), 'weather') \
                                  .agg(count('date').alias('count'))

yearly_weather_counts_pivot = yearly_weather_counts.groupBy('year') \
                                                  .pivot('weather') \
                                                  .agg(count('count').alias('count')) \
                                                  .fillna(0)

# Compute the percentage of rainy days for each year
yearly_rainy_perc = yearly_weather_counts_pivot.select('year', (when(col('rain') > 0, col('rain')) \
                                                                .otherwise(0) / sum('count')) \
                                                               .alias('percentage_rainy_days'))

# Print the result
yearly_rainy_perc.show()


AnalysisException: Column 'count' does not exist. Did you mean one of the following? [sun, fog, rain, snow, year, drizzle];
'Aggregate [year#1773, (CASE WHEN (rain#1776L > cast(0 as bigint)) THEN rain#1776L ELSE cast(0 as bigint) END / sum('count)) AS percentage_rainy_days#1786]
+- Project [coalesce(year#1712, cast(0.0 as int)) AS year#1773, coalesce(drizzle#1746L, cast(0.0 as bigint)) AS drizzle#1774L, coalesce(fog#1747L, cast(0.0 as bigint)) AS fog#1775L, coalesce(rain#1748L, cast(0.0 as bigint)) AS rain#1776L, coalesce(snow#1749L, cast(0.0 as bigint)) AS snow#1777L, coalesce(sun#1750L, cast(0.0 as bigint)) AS sun#1778L]
   +- Project [year#1712, __pivot_count(count) AS count AS `count(count) AS count`#1745[0] AS drizzle#1746L, __pivot_count(count) AS count AS `count(count) AS count`#1745[1] AS fog#1747L, __pivot_count(count) AS count AS `count(count) AS count`#1745[2] AS rain#1748L, __pivot_count(count) AS count AS `count(count) AS count`#1745[3] AS snow#1749L, __pivot_count(count) 